# Programmentwurf zu Aufgabe 1: Aussagenlogische Formel in Disjunkter Normalform als zweischichtiges Neuronales Netz

Autoren: Felix Luck, Jannes Süß, Julian Blönnigen, Phil Potratz

In [ ]:
import numpy as np

## Aufgabe (a) und (b): Implementierung des Netzes inklusive Dokumentation

In diesem Teil ist das wurde das neuronale Netz innerhalb von der Funktion training() implementiert. Hier gibt es außerdem die Aktivierungsfunktion sgn() und die Funktion create_data(), welche eine zufällige Formel in DNF erstellt inklusive zufälliger Belegungen. Wir haben zuletzt noch die Funktion summary() hinzugefügt, um ausgewählte Daten anschaulich darzustellen.

### Erstellung der Trainingsdaten
Hier werden die Trainingsdaten erstellt.

Am Anfang wird ein zweidimensionales Array mit zufälligen Belegungungen für die Variablen erstellt. Dieses Array hat die Größe n (Anzahl der Variablen) mal daten_umgfang (Anzahl der unterschiedlichen Belegungen).

Danach wird die Aussagenlogische Formel erstellt, welche aus einer Liste der Monome besteht. Die Monome bestehen wiederum aus den Indizies der Variablen. Wenn ein Index negativ ist bedeutet dies, dass dieses Literal negiert ist, hier wird dann als Index der Betrag des negativen Indexes genommen.

Zum Schluss werden die Wahrheitswerte der einzelnen Belegungen nochn generiert. Dafür wird eine Funktion genommen, welche jedes Monom auf den Wahrheitswert prüft. Wenn ein Monom wahr ist wird 1 zurückgegeben (Belegung ist wahr), sonst wird -1 zurückgegeben (Belegung ist falsch).

In [ ]:
def create_data(n, m, l, daten_umfang):
    # Erstellung von zufälligen Belegungen der Eingangsvariablen
    x_array = np.random.choice([-1, 1], size=(daten_umfang, n))

    # Erstellung der Aussagenlogischen Formel in DNF-Form
    monome = []
    for _ in range(m):
        monom = []
        
        # Zufällige, eindeutige Auswahl von Variablenindizes
        variablen = np.random.choice(n, size=l, replace=False)
        
        for v in variablen:
            # Zufällig entscheiden, ob normal oder negiert
            if np.random.rand() < 0.5:
                monom.append(v+1)   # Normal
            else:
                monom.append((v+1)*(-1))  # Negiert
        
        monome.append(monom)

    # Wahrheitswert der Aussagenlogischen Formel zurückgeben
    def aussagenlogische_formel(x):
        for monom in monome:
            alle_bedingungen_erfüllt = True
            
            for i in monom:
                index = abs(i)

                # Positive Variable (nicht negiert)
                if i >= 0:
                    if x[index-1] != 1:
                        alle_bedingungen_erfüllt = False
                        break
                # Negative Variable (negiert)
                else:
                    if x[index-1] != -1:
                        alle_bedingungen_erfüllt = False
                        break
            
            # Alle Variablen eines Monoms sind wahr
            if alle_bedingungen_erfüllt:
                return 1

        return -1  # Kein Monom erfüllt

    # Zielausgaben generieren
    p_array = np.array([aussagenlogische_formel(x) for x in x_array])

    return x_array, p_array, monome

## Aktivierungsfunktion
Als Aktivierungsfunktion wird die sgn()-Funktion genutzt, welche bei Werten größer und gleich null eins zurückgibt und sonst minus eins:

$
\operatorname{sgn}(x) = \begin{cases} 
1, & \text{falls } x \geq 0 \\
-1, & \text{falls } x < 0
\end{cases}
$

In [ ]:
def sgn(x):
    return np.where(x >= 0, 1, -1)

In [ ]:
# Training

def training(x_array, p_array, daten_umfang, epochs, eta, w, W, v, V):
    print("\n===== Training =====")
    for epoch in range(epochs):
        correct_predictions = 0
        sum_fehler = 0
        
        for x, p in zip(x_array, p_array):
            
            # Vorwärtspass
            z = sgn(np.dot(w, x) - v)
            y = sgn(np.dot(W, z) - V)

            # Fehler
            fehler = p - y
            sum_fehler += abs(fehler)

            # Gewichts-Updates (Fehler-Rückübertragung)
            delta_W = eta * fehler * z
            delta_w = eta * np.outer(W.flatten() * fehler, x)
            
            delta_V = -eta * fehler
            delta_v = -eta * fehler * W.flatten()

            # Aktualisierung
            w += delta_w
            W += delta_W
            v += delta_v
            V += delta_V

            if y == p:
                correct_predictions += 1
        
        # Ausgabe alle 1000 Epochen
        if epoch % (epochs // 10) == 0 or epoch == epochs - 1:
            acc = correct_predictions / daten_umfang
            print(f"Epoche {epoch+1}: Ergebnis: {correct_predictions}/{daten_umfang} ({acc:.2%}) Fehler: {sum_fehler}")

    print("Training abgeschlossen")

Hier wird eine Zusammenfassung der gewählten Konstanten und der Zufällig generierten Formel und deren Belegungen ausgegeben.

In [ ]:
def summary(n,m,l,daten_umfang, eta,epochs,x_array,p_array,monome):
    # Kleine Zusammenfassung

    # Konstannten
    print("===== Einstellungen =====")
    print(f"Anzahl Eingangsvariablen (n): {n}")
    print(f"Anzahl Monome (m): {m}")
    print(f"Anzahl Literale pro Monom (l): {l}")
    print(f"Anzahl Trainingsdaten: {daten_umfang}")
    print(f"Lernrate (eta): {eta}")
    print(f"Anzahl Epochen: {epochs}")

    # Aussagenlogische Formel
    print("\n===== Generierte DNF-Formel =====")
    dnf_teile = []
    for monom in monome:
        bedingungen = []
        for var in monom:
            if var >= 0:
                bedingungen.append(f"x{var}")
            else:
                bedingungen.append(f"¬x{abs(var)}")
        dnf_teile.append("(" + " ∧ ".join(bedingungen) + ")")

    dnf_formel = " ∨ ".join(dnf_teile)
    print(dnf_formel)

    # Teil der Belegungen
    print("\n===== Belegungen =====")
    for i in range(min(5, len(x_array))):
        print(f"[{i}]: {x_array[i]}  -->  [{p_array[i]}]")
    print("...")

## Aufgabe (c): Testen mit perfekten Gewichten und Schwellenwerten

Hier erstellen wir mithilfe der Funktion create_gewichte_und_schwellen() die Gewichte und Schwellenwerte, wie in den Anforderungen beschrieben, damit die DNF sofort implementiert wird. Danach wird trainiert und geschaut, ob sich die Gewichte und Schwellenwerte noch ändern.

In [ ]:
# Erstellung von theoretisch perfekten Gewichten und Schwellenwerten
def create_gewichte_und_schwellen(monome, n):
    """
    monome: Liste von Listen mit Literalen (positiv für x_i, negativ für ¬x_i)
    n: Gesamtanzahl der Variablen
    
    Rückgabe:
    - w: (m, n) Gewichtsmatrix für die Zwischenschicht
    - v: (m,) Schwellenwerte für die Zwischenschicht
    - W: (1, m) Gewichtsmatrix für die Ausgabeschicht
    - V: (1,) Schwellenwert für die Ausgabeschicht
    """
    m = len(monome)  # Anzahl der Monome
    
    w = np.zeros((m, n))
    v = np.zeros(m)
    
    for i, monom in enumerate(monome):
        for j, literal in enumerate(monom):
            if literal >= 0:
                w[i, literal-1] = 1.0
            else:
                w[i, abs(literal)-1] = -1.0
        v[i] = len(monom)  # Schwellenwert = Anzahl Literale im Monom
    
    W = np.ones((1, m))
    V = np.array([1 - m], dtype=float)  # Schwellenwert für die Ausgabeschicht
    
    return w, v, W, V

Die Ausgabe zeigt, dass es keinen Fehler gibt (p = y). Daher funktioniert die Implementierung.

In [ ]:
### Konstanten ###

# Anzahl Eingangsvariablen
n = 10

# Anzahl Monome
m = 5

# Anzahl Literale pro Monom
l = 10

# Anzahl Trainingsdaten
daten_umfang = 1000

# Lernrate
eta = 0.05

# Anzahl Epochen
epochs = 1000



### Daten generieren und Training starten ###

# Trainingsdaten generieren
x_array, p_array, monome = create_data(n, m, l, daten_umfang)

# Zusammenfassung der Daten
summary(n, m, l, daten_umfang, eta, epochs, x_array, p_array, monome)

# Perfekte Gewichte und Schwellenwerte erstellen
w, v, W, V = create_gewichte_und_schwellen(monome, n)

# Training starten
training(x_array, p_array, daten_umfang, epochs, eta, w, W, v, V)

## Aufgabe (d): Testen von anderen Gewichten und Schwellenwerten

### Test mit leicht veränderten Gewichten und Schwellenwerten

Am Anfang gibt es noch Fehler, dieser wird dann aber korrigiert.

In [ ]:
# Gewichte und Schwellenwerte leicht ändern

noise = 1
w_noise = w + np.random.uniform(-noise, noise, size=w.shape)
v_noise = v + np.random.uniform(-noise, noise, size=v.shape)
W_noise = W + np.random.uniform(-noise, noise, size=W.shape)
V_noise = V + np.random.uniform(-noise, noise, size=V.shape)

# Testen der Gewichte
training(x_array, p_array, daten_umfang, epochs, eta, w_noise, W_noise, v_noise, V_noise)

Zwar gibt es auch keine Fehler mehr, aber trotzdem sind die Gewichte und Schwellwerte nicht gleich zu denen, die theoretisch ermittelt wurden.

In [ ]:
# Vergleich der Gewichte und Schwellenwerte
print(f"w = w_noise: {np.array_equal(w, w_noise)}")
print(f"v = v_noise: {np.array_equal(v, v_noise)}")
print(f"W = W_noise: {np.array_equal(W, W_noise)}")
print(f"V = V_noise: {np.array_equal(V, V_noise)}")

### Test mit zufälligen Gewichten und Schwellenwerten

Das Ergebnis ist nie ganz perfekt. Die Accuracy nähert sich den 100% an, erreicht sie aber nie. Das liegt daran, dass die Fehlerrückübertragung in einem lokalen Minima liegt. Hier wird also gezeigt, das es für den Algorithmus schwieriger ist blind die DNF zu erlernen.

In [ ]:
# Zufällige Gewichte initialisieren

# Zufallsfakktor (Damit Werte nicht zu weit zerstreut sind)
zf = 0.1

w_random = np.random.randn(m, n) * zf
W_random = np.random.randn(1, m) * zf

v_random = np.random.randn(m) * zf
V_random = np.random.randn(1) * zf

# Testen der Gewichte
training(x_array, p_array, daten_umfang, epochs, eta, w_random, W_random, v_random, V_random)